In [2]:
import pandas as pd
import numpy as np
import seaborn as sns # For Data Visualization
import matplotlib.pyplot as plt # For Data Visualization

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [4]:
data=pd.read_csv("/content/drive/MyDrive/finance_complaint/consumer_complaints.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (5,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
data=data

In [6]:
data.head(3)

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473


In [7]:
data.loc[(data['consumer_complaint_narrative'].isnull()==False) & (data['company_public_response'].isnull()==False)].head(3)

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
190237,03/19/2015,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",NaN,Experian is reporting my OPEN and CURRENT Mort...,Company chooses not to provide a public response,Experian,CA,900XX,Older American,Consent provided,Web,03/25/2015,Closed with explanation,Yes,No,1290254
190263,03/19/2015,Credit card,NaN,Other,NaN,i opened XXXX Bank of America credit cards 15-...,Company chooses not to provide a public response,Bank of America,OR,971XX,NaN,Consent provided,Web,03/19/2015,Closed with monetary relief,Yes,Yes,1292209
191294,03/19/2015,Debt collection,Mortgage,False statements or representation,Impersonated an attorney or official,Lawyers at this law firm ( XXXX ) sent me a le...,Company believes it acted appropriately as aut...,"Dean Morris, L.L.P.",LA,708XX,NaN,Consent provided,Web,04/14/2015,Closed with explanation,Yes,Yes,1290181


In [8]:
df=data.drop(['issue','company_public_response','consumer_complaint_narrative','sub_issue','complaint_id','company_response_to_consumer','date_sent_to_company','zipcode','date_received'],axis=1)

In [9]:
df.head(3)

,product,sub_product,company,state,tags,consumer_consent_provided,submitted_via,timely_response,consumer_disputed?
0,Mortgage,Other mortgage,U.S. Bancorp,CA,NaN,NaN,Referral,Yes,Yes
1,Mortgage,Other mortgage,Wells Fargo & Company,CA,NaN,NaN,Referral,Yes,Yes
2,Credit reporting,NaN,Wells Fargo & Company,NY,NaN,NaN,Postal mail,Yes,No


In [19]:
df=df.sample(10000)
df.shape

(10000, 9)

In [20]:
train_X=df.drop('consumer_disputed?',axis=1)
train_y=df['consumer_disputed?']

In [21]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 82987 to 492012
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   product                    10000 non-null  object
 1   sub_product                7128 non-null   object
 2   company                    10000 non-null  object
 3   state                      9916 non-null   object
 4   tags                       1368 non-null   object
 5   consumer_consent_provided  2237 non-null   object
 6   submitted_via              10000 non-null  object
 7   timely_response            10000 non-null  object
 8   consumer_disputed?         10000 non-null  object
dtypes: object(9)
memory usage: 781.2+ KB


**Creating Pipeline**

In [22]:

from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

categorical_transformer=Pipeline(steps=[
            ('imputer',SimpleImputer(strategy='most_frequent')),
            ('onehot',OneHotEncoder(handle_unknown='ignore'))
])
#numerical_transformer=Pipeline(steps=[
          #    ('imputer', SimpleImputer(strategy='median'))                       
#])

preprocessor=ColumnTransformer(transformers=[
    ('cat',categorical_transformer,train_X.columns)
])


In [23]:
from sklearn.preprocessing import LabelEncoder

labels=df['consumer_disputed?']
labelencoder= LabelEncoder()
train_y=labelencoder.fit_transform(labels)

In [24]:
train_y

array([0, 0, 0, ..., 0, 0, 0])

**Normal Logistic Regression**

In [36]:
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

#model_ = XGBClassifier(n_estimators=2, max_depth=2, learning_rate=1, objective='binary:logistic')
model_=LogisticRegression()

# define models and parameters
solvers = ['newton-cg', 'lbfgs', 'liblinear']
penalty = ['l2']
c_values = [100, 10, 1.0, 0.1, 0.01]

k_fold = KFold(n_splits=3)

fin_pipeline=Pipeline([
    ('preprocessor',preprocessor),
     ("model", model_)
])
grid = dict(model__solver=solvers,model__penalty=penalty,model__C=c_values)

grid_search = GridSearchCV(estimator=fin_pipeline, param_grid=grid, n_jobs=-1, cv=k_fold, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(train_X, train_y)

#for train_indices,test_indices in k_fold.split(train_X, train_y):
  #print("TRAIN:", train_indices, "TEST:", test_indices)


In [38]:
# summarize results
print("Best score for Logistic Regression: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best score: 0.798305 using {'model__C': 0.01, 'model__penalty': 'l2', 'model__solver': 'newton-cg'}
0.795464 (0.013894) with: {'model__C': 100, 'model__penalty': 'l2', 'model__solver': 'newton-cg'}
0.796556 (0.014193) with: {'model__C': 100, 'model__penalty': 'l2', 'model__solver': 'lbfgs'}
0.795457 (0.013885) with: {'model__C': 100, 'model__penalty': 'l2', 'model__solver': 'liblinear'}
0.795723 (0.013914) with: {'model__C': 10, 'model__penalty': 'l2', 'model__solver': 'newton-cg'}
0.796371 (0.014114) with: {'model__C': 10, 'model__penalty': 'l2', 'model__solver': 'lbfgs'}
0.795722 (0.013913) with: {'model__C': 10, 'model__penalty': 'l2', 'model__solver': 'liblinear'}
0.796553 (0.014128) with: {'model__C': 1.0, 'model__penalty': 'l2', 'model__solver': 'newton-cg'}
0.796671 (0.014206) with: {'model__C': 1.0, 'model__penalty': 'l2', 'model__solver': 'lbfgs'}
0.796547 (0.014125) with: {'model__C': 1.0, 'model__penalty': 'l2', 'model__solver': 'liblinear'}
0.797024 (0.014510) with: {'model

**RidgeClassifier tuning alpha**

In [39]:
from sklearn.linear_model import RidgeClassifier

# define models and parameters
model_=RidgeClassifier()
alpha = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0]

k_fold = KFold(n_splits=3)

fin_pipeline=Pipeline([
    ('preprocessor',preprocessor),
     ("model", model_)
])
grid = dict(model__alpha=alpha)

grid_search = GridSearchCV(estimator=fin_pipeline, param_grid=grid, n_jobs=-1, cv=k_fold, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(train_X, train_y)

In [40]:
# summarize results
print("Best score for Ridge Classifier: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best score for Ridge Classifier: 0.795901 using {'model__alpha': 1.0}
0.795490 (0.013904) with: {'model__alpha': 0.1}
0.795610 (0.013927) with: {'model__alpha': 0.2}
0.795646 (0.013930) with: {'model__alpha': 0.3}
0.795720 (0.013934) with: {'model__alpha': 0.4}
0.795758 (0.013907) with: {'model__alpha': 0.5}
0.795821 (0.013893) with: {'model__alpha': 0.6}
0.795826 (0.013884) with: {'model__alpha': 0.7}
0.795862 (0.013892) with: {'model__alpha': 0.8}
0.795867 (0.013882) with: {'model__alpha': 0.9}
0.795901 (0.013889) with: {'model__alpha': 1.0}


**KNN with hyperparameter training**

In [25]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold

## define model and hyperparameters
model_=KNeighborsClassifier()

neighbours=range(1,20,2)
wieghts=['uniform','distance']
metric = ['euclidean', 'manhattan', 'minkowski']

k_fold = KFold(n_splits=3)

param_grid=dict(model__n_neighbors=neighbours,model__weights=wieghts,model__metric=metric)

model_pipeline=Pipeline([
    ('preprocessor',preprocessor),
     ("model", model_)
])
grid_search=GridSearchCV(estimator=model_pipeline, param_grid=param_grid, n_jobs=-1, cv=k_fold, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(train_X, train_y)

In [26]:
# summarize results
print("Best score for KNN Classifier: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best score for KNN Classifier: 0.793301 using {'model__metric': 'euclidean', 'model__n_neighbors': 19, 'model__weights': 'uniform'}
0.686601 (0.006933) with: {'model__metric': 'euclidean', 'model__n_neighbors': 1, 'model__weights': 'uniform'}
0.686601 (0.006933) with: {'model__metric': 'euclidean', 'model__n_neighbors': 1, 'model__weights': 'distance'}
0.735001 (0.005056) with: {'model__metric': 'euclidean', 'model__n_neighbors': 3, 'model__weights': 'uniform'}
0.730001 (0.004621) with: {'model__metric': 'euclidean', 'model__n_neighbors': 3, 'model__weights': 'distance'}
0.756900 (0.002268) with: {'model__metric': 'euclidean', 'model__n_neighbors': 5, 'model__weights': 'uniform'}
0.748400 (0.003368) with: {'model__metric': 'euclidean', 'model__n_neighbors': 5, 'model__weights': 'distance'}
0.771401 (0.004848) with: {'model__metric': 'euclidean', 'model__n_neighbors': 7, 'model__weights': 'uniform'}
0.759201 (0.004879) with: {'model__metric': 'euclidean', 'model__n_neighbors': 7, 'model

**Support vector machine**

In [28]:
from sklearn.svm import SVC

## define model and hyperparameters
model_=SVC()

kernel=['linear', 'poly', 'rbf']
C=[50, 10, 1.0, 0.1, 0.01]
gamma=['scale']
k_fold = KFold(n_splits=3)
param_grid=dict(model__kernel=kernel,model__C=C,model__gamma=gamma)

model_pipeline=Pipeline([
    ('preprocessor',preprocessor),
     ("model", model_)
])

grid_search=GridSearchCV(estimator=model_pipeline, param_grid=param_grid, n_jobs=-1, cv=k_fold, scoring='accuracy',error_score=0)
grid_result = grid_search.fit(train_X, train_y)

In [29]:
# summarize results
print("Best score for SVC Classifier: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best score for SVC Classifier: 0.795401 using {'model__C': 1.0, 'model__gamma': 'scale', 'model__kernel': 'poly'}
0.785901 (0.007253) with: {'model__C': 50, 'model__gamma': 'scale', 'model__kernel': 'linear'}
0.719400 (0.001327) with: {'model__C': 50, 'model__gamma': 'scale', 'model__kernel': 'poly'}
0.725400 (0.002599) with: {'model__C': 50, 'model__gamma': 'scale', 'model__kernel': 'rbf'}
0.785901 (0.007253) with: {'model__C': 10, 'model__gamma': 'scale', 'model__kernel': 'linear'}
0.747000 (0.004732) with: {'model__C': 10, 'model__gamma': 'scale', 'model__kernel': 'poly'}
0.763800 (0.001693) with: {'model__C': 10, 'model__gamma': 'scale', 'model__kernel': 'rbf'}
0.786501 (0.006838) with: {'model__C': 1.0, 'model__gamma': 'scale', 'model__kernel': 'linear'}
0.795401 (0.005991) with: {'model__C': 1.0, 'model__gamma': 'scale', 'model__kernel': 'poly'}
0.795301 (0.005902) with: {'model__C': 1.0, 'model__gamma': 'scale', 'model__kernel': 'rbf'}
0.795301 (0.005902) with: {'model__C': 0.1,

In [30]:
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

#kfold = KFold(3, True, 1)
k_fold = KFold(n_splits=3)
#cross_val_score(lasso, X, y, cv=3)
X = np.array([[1, 2,7,4], [3, 4,6,2], [8,12,1, 2], [3,5,7, 4]])
y = np.array([0, 1, 0, 1])
#rskf = RepeatedStratifiedKFold(n_splits=2, n_repeats=2,    random_state=36851234)
for train_index, test_index in k_fold.split(X, y):
  print("TRAIN:", train_index, "TEST:", test_index)
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]

TRAIN: [2 3] TEST: [0 1]
TRAIN: [0 1 3] TEST: [2]
TRAIN: [0 1 2] TEST: [3]


In [24]:
X

array([[ 1,  2,  7,  4],
       [ 3,  4,  0,  2],
       [ 8, 12,  1,  2],
       [ 3,  5,  7,  4]])